In [5]:
# import necessary imports
import numpy as np
from numpy_ml.neural_nets.optimizers import Adam


In [28]:
# data preprocessing 
    # load encoded train/validation/test sets
    # chunken und batchen

batch_size = 1
chunk_size = 1

def get_batch(input, batch_size):
    return np.array([[1,2,3], [3,4,7]]), np.array([[1,2,3], [3,4,7]]) # pls adapt. obv. :))))))

In [ ]:
class neural_embedding:
    def __init__(self, vocab_size):
        # they call nn.Embedding, need to look up what that is, exactly
        self.vocab_size = vocab_size
        self.token_embedding_table = np.random.rand(vocab_size, vocab_size)

    def calculate_softmax(self, x):
        """Takes input array x and returns softmax."""
        soft_x = np.exp(x - np.max(x))
        softer_x = soft_x / np.sum(soft_x)
        return softer_x
    

    def calculate_cross_entropy(self, y_hatless, y_hat):
        """
        Takes target (y_hatless) and prediction (y_hat) and computes cross entropy loss.
        """
        # get vocab_size
        _, _, vocab_size = y_hat.shape        
        y_hat = y_hat.reshape(y_hat.shape[0]*y_hat.shape[1], y_hat.shape[2])
        y_hatless_flat = y_hatless.reshape(-1)
        # one-hot encode targets
        y_hatless_hot = np.eye(vocab_size)[y_hatless_flat]
       
        y_hat = self.calculate_softmax(y_hat)
    
        return -np.sum(y_hatless_hot*np.log(y_hat))
    
    def forward(self, idx, targets=None):
        """
        Implements forward pass with an unnecessary logitte function 
        which i only did not delete because now I'm emotionally attached.
        Args:
            idx(np.array): (B,T) numpy array of integers
            targets(np.array): (B,T) numpy array of integers
        Returns:
            input_logits(np.array)
            sometimes also: targets(np.array)
        """
        batch_size, chunk_size = idx.shape
        logits = np.zeros((batch_size, chunk_size, (self.token_embedding_table[0].size)))

        def logitte(batch_size, chunk_size, input):
            for batch in range(batch_size):
                for chunk in range(chunk_size):
                    # (B,T,C) b=batch_size, t="time"=chunk_size, c=vocab_size
                    logits[batch][chunk] = self.token_embedding_table[input[batch][chunk]]
                    
            return logits

        input_logits = logitte(batch_size, chunk_size, idx)
        
        if targets is not None:
            loss = self.calculate_cross_entropy(targets, input_logits)

            return input_logits, loss

        return input_logits

    
    def backward(self, targets, input_logits):
        # need to do the same reshaping as we did for cross entropy, apaprently
        targets_flat = targets.reshape(-1)
        one_hot_targets = np.eye(self.vocab_size)[targets_flat]

        # shape after: ((batch_size*chunk_size), vocab_size)
        input_logits_2d = input_logits.reshape(input_logits.shape[0]*input_logits.shape[1], input_logits.shape[2])

        # somehow this is supposedly the combiantion of the derivative of softmax with the derivative of the CCE
        delta = one_hot_targets - input_logits_2d
        # compute gradient for weight matrix: dot product between the transpose of the to layer and delta vector computed above
        # conversations have resulted in might only need to multiply this with the part of token embedding table 
        # part that is actually used should the have size (6,80) so np.dot((80,6),(6,80)) should then work?
        # or maybe i'm just supposed to add them? We'll see tomorrow.
        gradient = np.dot(delta.transpose(),self.token_embedding_table)
        # would then need to fill up the rest with gradients of 0 so that I get 80,80 in the end?
    
        return gradient

    

In [31]:
# currently do not use loss at all, so something is probably very wrong

def train(model, optimiser):
    batch_size=32
    for steps in range(2): # TODO: please increase
        # sample batch of data
        xb, yb = get_batch('train', batch_size) # TODO: pls adapt to above fct

        # get logits and loss
        logits, loss = model.forward(xb, yb)
        print(f"loss: {loss}")

        gradient = model.backward(yb, logits)
        optimiser.update(param=model.token_embedding_table, param_grad=gradient, param_name="gradient")
        

In [32]:
vocab_size = 80

my_neural_embedding = neural_embedding(vocab_size)
optimiser = Adam(my_neural_embedding.token_embedding_table)
train(my_neural_embedding, optimiser)
#uglies_array_youve_ever_seeeeeen = np.array([[1,2,3], [3,4,7]])
#my_input_logits, my_loss = my_neural_embedding.forward(uglies_array_youve_ever_seeeeeen, uglies_array_youve_ever_seeeeeen)




loss: 36.03990560393165


ValueError: shapes (80,6) and (80,80) not aligned: 6 (dim 1) != 80 (dim 0)